In [1]:
from IPython.display import HTML

# necessary for scrolling horizontally
HTML("""
<style>
    .output_scroll {overflow-x: auto;}
</style>
""")

In [2]:
from src.agent import Agent

It is also possible to run the Agent without reranking at all, but then a lot of irrelevant documents are extracted and sent to an LLM, which is undesirable. Therefore, only the following experiments will be run:

1. Semantic retrieval and reranking with cross encoder only (without Elasticsearch).
2. Semantic retrieval and reranking with cross encoder and Elasticsearch scores. Their scores will be fused with weights according to the formula: $$\text{score} = \alpha \cdot \text{el\_score} + (1 - \alpha) \cdot \text{ce\_score},$$ where $\text{el\_score}$ is an Elasticsearch score, $\text{ce\_score}$ is a cross encoder. Both scores are squashed into $[0, 1]$ interval, so the resulting score also belongs to that interval. This experiment will be conducted for $\alpha = 0.7$.
3. Semantic retrieval and reranking with Elasticsearch scores only. In this case, the scores will be fused as in the previous experiment but instead of reranking scores cosine similarity of the retrieved documents will be used. Once again, $\alpha = 0.7$.

In all the experiments, the documents with a score between $0.2$ and $0.9$ will be checked by the LLM and updated if necessary, and all the documents with a score above $0.9$ are updated by the LLM right away. Of course, one can just increase the confidence threshold in order to check more and update (automatically) less documents.

## 1. Semantic retrieval and reranking with cross encoder only (without Elasticsearch)

In [3]:
config = {
    "embedding_model_name": "text-embedding-3-small",
    "llm_model_name": "gpt-4o-mini",
    "dim": 1536,
    "reranker_model_name": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "use_elastic_search": False,
    "use_cross_encoder_reranking": True,
}

In [3]:
def create_agent(config_dict):
    agent = Agent(**config_dict)
    return agent

In [5]:
from IPython.display import clear_output

query = "We removed the ability to archive queries, and instead added the ability to completely delete them. Update all relevant knowledge"
agent = create_agent(config)
await agent.run(query)
clear_output(wait=True)
agent.print_tree()

Pipeline Execution
┣━━ Step 1: Query Analysis and Rewriting
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Initial user query:                                                                                     │
┃   ┃   │ We removed the ability to archive queries, and instead added the ability to completely delete them.     │
┃   ┃   │ Update all relevant knowledge                                                                           │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Feature changed:                                                                                        │
┃   ┃   │ ability to archive queries                                                                              │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Identified changes:                                                                                     │
┃   ┃   │ removed the ability to archive queries and added the ability to completely delete them                  │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┗━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃       │ Rewritten query:                                                                                        │
┃       │ Feature ability to archive queries was changed: removed the ability to archive queries and added the    │
┃       │ ability to completely delete them                                                                       │
┃       ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┗━━ Step 2: Extraction of Relevant Documents
    ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
    ┃   │ The vector store is populated.                                                                          │
    ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
    ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
    ┃   │ Retrieval hyperparameters:                                                                              │
    ┃   │ Cutoff cosine similarity score: 0.1                                                                     │
    ┃   │ Cutoff reranker score: 0.2                                                                              │
    ┃   │ Reranker confidence score: 0.9.                                                                         │
    ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
    ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
    ┃   │ Extracted 1023 documents in total with range search. Scores are ranging from 0.1 to 0.46.               │
    ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
    ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
    ┃   │ Retrieved documents have been reranked: scores are in the range: [1.1211369e-05, 0.10182693]            │
    ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
    ┗━━ ╭──────────────────────────────────────────────────────────

As we can see, semantic search with reranking is not very effective. It produces very low scores which prevents from setting a meaningful threshold. Let's try combining it with Elasticsearch.

## 2. Semantic retrieval and reranking with cross encoder and Elasticsearch scores.

In [4]:
config_elastic = {
    "embedding_model_name": "text-embedding-3-small",
    "llm_model_name": "gpt-4o-mini",
    "dim": 1536,
    "reranker_model_name": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "use_elastic_search": True,
    "use_cross_encoder_reranking": True,
    "weight_elastic": 0.7
}

In [5]:
from IPython.display import clear_output
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

load_dotenv()

# Clear Elasticsearch index (in case it was updated in the previous run)
es = Elasticsearch(os.environ.get('ELASTICSEARCH_URL'))
index_name = "documents"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Index '{index_name}' deleted.")
    clear_output(wait=True)

query = "We removed the ability to archive queries, and instead added the ability to completely delete them. Update all relevant knowledge"
agent = create_agent(config_elastic)
await agent.run(query)
clear_output(wait=True)
agent.print_tree()

Pipeline Execution
┣━━ Step 1: Query Analysis and Rewriting
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Initial user query:                                                                                     │
┃   ┃   │ We removed the ability to archive queries, and instead added the ability to completely delete them.     │
┃   ┃   │ Update all relevant knowledge                                                                           │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Feature changed:                                                                                        │
┃   ┃   │ ability to archive queries                                                                              │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Identified changes:                                                                                     │
┃   ┃   │ We removed the ability to archive queries, and instead added the ability to completely delete them.     │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┗━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃       │ Rewritten query:                                                                                        │
┃       │ Feature ability to archive queries was changed: We removed the ability to archive queries, and instead  │
┃       │ added the ability to completely delete them.                                                            │
┃       ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┣━━ Step 2: Extraction of Relevant Documents
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ The vector store is populated.                                                                          │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Retrieval hyperparameters:                                                                              │
┃   ┃   │ Cutoff cosine similarity score: 0.1                                                                     │
┃   ┃   │ Cutoff reranker score: 0.2                                                                              │
┃   ┃   │ Reranker confidence score: 0.9.                                                                         │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Extracted 1002 documents in total with range search. Scores are ranging from 0.1 to 0.45.               │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Retrieved documents have been reranked: scores are in the range: [3.7742795939266223e-06,               │
┃   ┃   │ 0.6979256899999999]                                                                                     │
┃   ┃   ╰──────────────────────────────────────────────────────────

### Updates from the 'update' group

In [6]:
agent.display_changes(group='update')

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ No documents from the group update have been changed.                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Updates from the 'check' group

In [7]:
agent.display_changes(group='check')

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── CHANGES FROM THE CHECK GROUP ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

DOC ID: 1009

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── Diff ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   1 --- Before                                                                                                                                                                                                                                                                                                             │
│   2 +++ After                                                                                                                                                                                                                                                                                                              │
│   3 @@ -2,4 +2,4 @@                                                                                                                                                                                                                                                                                                        │
│   4   Queries on Dune need to be triggered and will be executed on one of our query engines.                                                                                                                                                                                                                               │
│   5   Query executions can be triggered by any user/team in Dune for public queries and by the owner of the query for private queries. Query executions can be triggered through the query editor, the dashboard editor, or via the API. We also automatically refresh popular dashboards automatically in regular         │
│     intervals.                                                                                                                                                                                                                                                                                                             │
│   6   The query engine determines the amount of resources allocated to your query. The larger the query engine, the more resources are allocated to your query. This means that queries executed on a larger query engine will run faster and are less likely to time out.                                                 │
│   7 -                                                                                                                                                                                                                                                                                                                      │
│   8 + Note: The ability to archive queries has been removed. Instead, users can now completely delete their queries.                                                                                                                                                                                                       │
╰────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

DOC ID: 199

╭───────────────────────────────────────────────────── Diff ─────────────────────────────────────────────────────╮
│    1 --- Before                                                                                                │
│    2 +++ After                                                                                                 │
│    3 @@ -8,6 +8,7 @@                                                                                           │
│    4  allow weekly or monthly unique users to be calculated incrementally by                                   │
│    5  combining the dailies. This is similar to computing weekly revenue by                                    │
│    6  summing daily revenue. Uses of  approx_distinct  with  GROUPING SETS  can be converted to use            │
│      HyperLogLog .                                                                                             │
│    7 + Note: The ability to archive queries has been removed, and users can now completely delete queries      │
│      instead.                                                                                                  │
│    8   Examples:                                                                                               │
│    9      CREATE   TABLE  visit_summaries  (                                                                   │
│   10       visit_date  date ,                                                                                  │
╰────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── Diff ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   1 --- Before                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       │
│   2 +++ After                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        │
│   3 @@ -1,2 +1,2 @@                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

DOC ID: 406

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── Diff ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│    1 --- Before                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      │
│    2 +++ After                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       │
│    3 @@ -9,4 +9,6 @@                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

DOC ID: 1067

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── Diff ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│    1 --- Before                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      │
│    2 +++ After                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       │
│    3 @@ -7,8 +7,7 @@                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

As we can see, reranking with Elasticsearch helps to identify relevant documents. Now let's see what happens if we remove cross encoder from reranking process.

## 3. Semantic retrieval and reranking with Elasticsearch scores only.

In [4]:
config_elastic_only = {
    "embedding_model_name": "text-embedding-3-small",
    "llm_model_name": "gpt-4o-mini",
    "dim": 1536,
    "use_elastic_search": True,
    "use_cross_encoder_reranking": False,
    "weight_elastic": 0.7
}

In [7]:
from IPython.display import clear_output
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

load_dotenv()

# Clear Elasticsearch index (in case it was updated in the previous run)
es = Elasticsearch(os.environ.get('ELASTICSEARCH_URL'))
index_name = "documents"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Index '{index_name}' deleted.")
    clear_output(wait=True)

query = "We removed the ability to archive queries, and instead added the ability to completely delete them. Update all relevant knowledge"
agent = create_agent(config_elastic_only)
await agent.run(query)
clear_output(wait=True)
agent.print_tree()

Pipeline Execution
┣━━ Step 1: Query Analysis and Rewriting
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Initial user query:                                                                                     │
┃   ┃   │ We removed the ability to archive queries, and instead added the ability to completely delete them.     │
┃   ┃   │ Update all relevant knowledge                                                                           │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Feature changed:                                                                                        │
┃   ┃   │ ability to archive queries                                                                              │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Identified changes:                                                                                     │
┃   ┃   │ removed the ability to archive queries and added the ability to completely delete them                  │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┗━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃       │ Rewritten query:                                                                                        │
┃       │ Feature ability to archive queries was changed: removed the ability to archive queries and added the    │
┃       │ ability to completely delete them                                                                       │
┃       ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┣━━ Step 2: Extraction of Relevant Documents
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ The vector store is populated.                                                                          │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Retrieval hyperparameters:                                                                              │
┃   ┃   │ Cutoff cosine similarity score: 0.1                                                                     │
┃   ┃   │ Cutoff reranker score: 0.2                                                                              │
┃   ┃   │ Reranker confidence score: 0.9.                                                                         │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Extracted 1023 documents in total with range search. Scores are ranging from 0.1 to 0.46.               │
┃   ┃   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
┃   ┣━━ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
┃   ┃   │ Retrieved documents have been reranked: scores are in the range: [0.030088470876216893,                 │
┃   ┃   │ 0.8106449605869674]                                                                                     │
┃   ┃   ╰──────────────────────────────────────────────────────────

### Updates from the 'update' group

In [8]:
agent.display_changes(group='update')

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ No documents from the group update have been changed.                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Updates from the 'check' group

In [9]:
agent.display_changes(group='check')

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── CHANGES FROM THE CHECK GROUP ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

DOC ID: 1009

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── Diff ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   1 --- Before                                                                                                                                                                                                                                                                                                             │
│   2 +++ After                                                                                                                                                                                                                                                                                                              │
│   3 @@ -2,4 +2,4 @@                                                                                                                                                                                                                                                                                                        │
│   4   Queries on Dune need to be triggered and will be executed on one of our query engines.                                                                                                                                                                                                                               │
│   5   Query executions can be triggered by any user/team in Dune for public queries and by the owner of the query for private queries. Query executions can be triggered through the query editor, the dashboard editor, or via the API. We also automatically refresh popular dashboards automatically in regular         │
│     intervals.                                                                                                                                                                                                                                                                                                             │
│   6   The query engine determines the amount of resources allocated to your query. The larger the query engine, the more resources are allocated to your query. This means that queries executed on a larger query engine will run faster and are less likely to time out.                                                 │
│   7 -                                                                                                                                                                                                                                                                                                                      │
│   8 + Note: The ability to archive queries has been removed. Users can now completely delete queries instead.                                                                                                                                                                                                              │
╰────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

DOC ID: 199

╭──────────────────────────────────────────────── Diff ─────────────────────────────────────────────────╮
│   1 --- Before                                                                                        │
│   2 +++ After                                                                                         │
│   3 @@ -22,4 +22,4 @@                                                                                 │
│   4      FROM  visit_summaries                                                                        │
│   5      WHERE  visit_date  >=   current_date   -   interval   '7'   day ;                            │
│   6                                                                                                   │
│   7 -                                                                                                 │
│   8 + Note: The ability to archive queries has been removed, and users can now completely delete      │
│     queries.                                                                                          │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── Diff ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   1 --- Before                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       │
│   2 +++ After                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        │
│   3 @@ -1,2 +1,2 @@                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

DOC ID: 406

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── Diff ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   1 --- Before                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       │
│   2 +++ After                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        │
│   3 @@ -9,4 +9,4 @@                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

DOC ID: 1067

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── Diff ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   1 --- Before                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       │
│   2 +++ After                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        │
│   3 @@ -1 +1 @@                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    